Code pour la génération des résultats et images de l'article

In [ ]:
import numpy as np
import scipy, os, pickle, multiprocessing
import matplotlib.pyplot as plt
import sklearn
import sklearn.cluster as skc
import cvxpy as cp
import scipy, re
import pandas as pd

%matplotlib qt

## Fonctions

In [13]:
def algoMaxscore(params, fonc, solspec, nit):
    #Algo 2 du papier
    # params : paramètres de la fonction objectif (matrice d'adjacence observée, autres paramètres p et q)
    # fonc : une fonction objectif à maximiser
    # solspec : solutions (listes de labels ) proposées par la méthode spectrale
    # nit : nombre itérations
    # ** renvoie la solution (liste de labels) maximisant `fonc` 
    solmax, fmax = None, -float('inf')
    for sol_init in solspec:
        sol, fsol, flabels = gibbsSampler([sol_init.copy(), *params], fonc, nit=nit)
        if fsol>fmax:
            fmax=fsol
            solmax=sol
    return solmax

def algoSpectral(matbs, taillemax=10):
    #Algo Shi & Malik 2000.  Partitionne avec Kmeans chaque matrice sur 2 à $taillemax classes.
    # matbs : liste des matrices d'adjacence à partitionner
    # taillemax : nombre maximal de classes.
    # renvoie : listes de labels trouvés par Kmeans. Il y a $taillemax-2 propositions par matrice, chacune proposant un nombre différent de classes.
    nbclus=min(taillemax, matbs[0].shape[0]+1)
    solspec = np.empty([len(matbs), nbclus-2, matbs[0].shape[0]])
    for h,mat in enumerate(matbs):
        diag=np.sum(mat, axis=1)
        lapl=(np.diag(diag)-mat)/diag[:, np.newaxis]#calcul du laplacien
        valps, vecps = np.linalg.eig(lapl)
        valps, vecps = np.real(valps), np.real(vecps)
        vecps_tries = vecps[:, np.argsort(valps)] # trier les vecteurs propres en fonction des valeurs propres
        retireprem=1*(np.var(vecps_tries[:, 0])<0.15*np.var(vecps_tries[:, 1])) # ignore le premier vecteur propre si sa variance est trop faible (il ne contient pas d'information permettant d'identifier les classes)
        for i in range(2, nbclus):
            algocls=skc.KMeans(i)
            solspec[h,i-2] = algocls.fit_predict(vecps_tries[:, retireprem:i])
    return solspec


# Parallélisation des calculs pour un (grand) nombre de matrices d'adjacence.
def calcExperience(i):
    mat, solspec = lesmatbs[i], lessolspecs[i]
    data={}
    resultat_spec=solspec[nbclustheo-2]
    data['spectral_brut'] = resultat_spec
    data['spectral_ari'] = sklearn.metrics.cluster.adjusted_rand_score(leslabels, resultat_spec)
    for fonc in lalistefoncs:
        resultat=algoMaxscore([mat, *getparamat(i)], fonc, solspec, nit=nit)
        data[fonc.__name__+'_brut'] = resultat
        data[fonc.__name__+'_ari'] = sklearn.metrics.cluster.adjusted_rand_score(leslabels, resultat)
    return data

def initializer(listefoncs, labels, paramats, matbs, solspecs):
    global lalistefoncs, leslabels, nbclustheo, theogroupes, nit, lesparamats, getparamat, lesmatbs, lessolspecs
    nit = 20
    leslabels = labels
    nbclustheo = len(set(labels))
    theogroupes =groupeSol(leslabels)
    lesparamats=paramats
    lalistefoncs=listefoncs
    lesmatbs=matbs
    lessolspecs=solspecs
    if paramats is None:
        getparamat = lambda i: (None, None)
    elif len(paramats)==1:
        getparamat = lambda i: lesparamats[0]
    else:
        getparamat = lambda i: lesparamats[i]
        
def teste_foncs_objectifs(matbs, listefoncs, labels, dparamats=None):
    #calculs de partitions
    # matbs : liste de matrices d'adjacence à partitionner
    # listefoncs : liste de fonctions objectifs à maximiser pour trouver le partitionnement optimal
    # labels : partition attendue. Supposée la même pour toutes les matrices
    # dparamts : liste des paramètres de génération des matrices. 
    solspecs = algoSpectral(matbs)
    with multiprocessing.Pool(processes=7, initializer=initializer, initargs=(listefoncs, labels, dparamats, matbs, solspecs)) as pool:
        ress = pool.map(calcExperience, list(range(len(matbs))))
    return pd.DataFrame.from_records(ress)

## Modèle global - Planted partition model

### Exemple

In [ ]:
blocliste=[0, 72, 24, 72] #tailles des clusters de la diagonale
mat=construit(blocliste) #matrice d'adjacence théorique
labels=labelblocs(blocliste) #labels associés aux nœuds

#paire des matrices (,q)
# telle que p est associée à la probabilité de présence d'un lien entre deux nœuds théoriquement liés
# et q est la probabilité de présence d'un lien entre deux nœuds théoriquement non liés

dparamat=pmats(sum(blocliste), p=0.7, q=0.3)
paramat=pmat_labels(labels, *dparamat)

#création d'une matrice d'adjacence bruitée
matb=brmat(paramat, graine=None)

#Partitionne la matrice $matb avec les différentes méthodes (spectrale, maximum de vraisemblance, moduarité).
#Les résultats sont contenus dans une DataFrame pandas
pddat = teste_foncs_objectifs([matb], [vraiss, modularite], labels, [dparamat])

In [16]:
pddat

,spectral_brut,spectral_ari,vraiss_brut,vraiss_ari,modularite_brut,modularite_ari
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0


In [ ]:
#affichage matrice des paramètres de la loi de Bernoulli
plt.figure(figsize=(5,5.2))
plt.imshow(paramat, vmin=0, vmax=1)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/simulee_p07q03_param.pdf")

In [ ]:
#affiche une matrice d'adjacence suivant le modèle prédéfini
plt.figure(figsize=(5,5.2))
plt.imshow(matb, vmin=0, vmax=1)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/simulee_p07q03_obs.pdf")

### Simulations

Tests des méthodes spectrale, modularité et maximum de vraisemblance sur des matrices générées suivant le "planted model"

In [ ]:
nb_simus=2#60 dans le papier
q=0.3
pvals=np.arange(0.2, 0.9,0.03)

blocliste=[0, 72, 24, 72]
mat=construit(blocliste)
labels=labelblocs(blocliste)

In [ ]:
#calcul des performances des différents algorithmes sur des matrices simulées avec différents paramètres p. 
for p in pvals:
    fic=f'calculs/modelepq_q{q:.2f}_p{p:.2f}.pickle'
    if os.path.exists(fic):
        continue
    paramats = pmats(sum(blocliste), p=p, q=q)
    matbs=[brmat(pmat_labels(labels, *paramats), graine=j) for j in range(nb_simus)]
    pddat = teste_foncs_objectifs(matbs, [vraiss, modularite], labels, [paramats])
    pddat.to_pickle(fic)

In [17]:
#Collecte les résultats précédemment obtenus, calcule le score moyen en fonction du paramètre p. 
fics = set([glob.path for glob in os.scandir('calculs') if glob.name.startswith('modelepq_')])
pattern = r'p([\d.]+)'
series = []
for fic in fics:
    pddat=pd.read_pickle(fic)
    ser=pddat[[col for col in pddat.columns if not col.endswith('brut')]].mean()
    ser['p']=float(re.search(pattern, fic.rstrip('.pickle')).group(1))
    series.append(ser)

#base de donnée des performances moyennes
finale = pd.concat(series, axis=1).T.sort_values(by='p')

In [19]:
plt.close('all')
ax = finale.plot('p', [col for col in finale.columns if col.endswith('ari')], figsize=(10, 5), ylabel='ARI moyen')

plt.tight_layout()
#plt.savefig('images/simulee_q03_resultats_ari.pdf')

## Modèle par nœuds

### Exemple

In [ ]:
blocliste=[0, 72, 24, 72]
mat=construit(blocliste)
labels=labelblocs(blocliste)

def paramnds(blocliste, ecartmax=0.4, pmin=0.1, pmax=0.9, graine=None):
    #fonction de génération de paramètres p,q aléatoires pour le modèle par nœuds
    probas=np.empty((2, sum(blocliste)))
    rng=np.random.default_rng(graine)
    probas[1] = rng.uniform(pmin, pmax, sum(blocliste))
    probas[0] = rng.uniform(np.maximum(pmin, probas[1]-ecartmax), np.minimum(pmax, probas[1]+ecartmax), sum(blocliste))

    #trie pour s'assurer que probas[0] (qui est la liste des q_n) est inférieur en chaque valeur à probas[1] (la liste des p_n)
    probas.sort(axis=0)

    #trie les nœuds en fonction de p_n. Cela permet de mieux identifier les clusters visuellement.
    cumblocliste=np.cumsum(np.concatenate(([0], blocliste)))
    for deb, fin in zip(cumblocliste, cumblocliste[1:]):
        sorted_indices = np.argsort(-probas[1, deb:fin])#trie selon p, ordre descendant
        probas[:, deb:fin] = probas[:, deb+sorted_indices]
    return probas

qns, pns = paramnds(blocliste, ecartmax=0.3, pmin=0.1, pmax=0.9, graine=None) 
dparamat=pmats(sum(blocliste), p=pns, q=qns)
paramat=pmat_labels(labels, *dparamat)
matb=brmat(paramat, graine=None)


In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(paramat, vmin=0, vmax=1)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/simulee_pqnds_param.pdf")

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(matb, vmin=0, vmax=1)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/simulee_pqnds_obs.pdf")

### Simulations

In [ ]:
nb_simus=2#60
pmin, pmax= 0.1, 0.9
evals=np.arange(0.1, 0.7, 0.03)#valeurs prises par e, l'écart max entre p_n et q_n

blocliste=[0, 72, 24, 72]
mat=construit(blocliste)
labels=labelblocs(blocliste)

In [ ]:
#calcul des performances des différents algorithmes sur des matrices simulées avec différents paramètres e. 
for e in evals:
    fic=f'calculs/modelpqn_e{e:.2f}.pickle'
    if os.path.exists(fic):
        continue
    desparamnoeuds = np.array([paramnds(blocliste, ecartmax=e, pmin=pmin, pmax=pmax, graine=i) for i in range(nb_simus)])
    dparamats=[pmats(sum(blocliste), p=elt[1], q=elt[0]) for elt in desparamnoeuds]
    matbs=[brmat(pmat_labels(labels, *dparamat), graine=j) for j,dparamat in enumerate(dparamats)]
    pddat = teste_foncs_objectifs(matbs, [vraiss, modularite], labels, [paramats])
    pddat.to_pickle(fic)

In [20]:
#Collecte les résultats précédemment obtenus, calcule le score moyen en fonction du paramètre e. 
fics = set([glob.path for glob in os.scandir('calculs') if glob.name.startswith('modelepqn_')])

pattern = r'e([\d.]+)'
series = []
for fic in fics:
    pddat=pd.read_pickle(fic)
    ser=pddat[[col for col in pddat.columns if not col.endswith('brut')]].mean()
    ser['e']=float(re.search(pattern, fic.rstrip('.pickle')).group(1))
    series.append(ser)
finale = pd.concat(series, axis=1).T.sort_values(by='e')

In [21]:
plt.close('all')
ax = finale.plot('e', [col for col in finale.columns if col.endswith('ari')], figsize=(10, 5), ylabel='ARI')

plt.tight_layout()
#plt.savefig("images/simulee_pqn_resultats_ari.pdf")
plt.savefig("images/simulee_pqn_resultats_ari.pdf")

## Experiences

Les matrices d'adjacences sont issues de test de présence de goulot entre des flux de paquets dans des réseaux émulées.
Ces flots ont été ordonnés connaissant la vérité terrain. Les matrices d'adjacence théoriques sont identiques à celle présentée dans les données simulées.

### Experience 1

Expérience non présentée dans l'article. Il suffit de voir la matrice pour identifier les goulots.

In [22]:
matb=np.load('donnees/experience1_facile.npy')#charge les résultats
blocliste=[0, 72, 24, 72]
mat=construit(blocliste)
labels=labelblocs(blocliste)


In [ ]:
#résolution avec la méthode spectrale et la modularité. Le calcul de la vraisemblance n'est pas possible car les paramètres sont inconnus
pddat = teste_foncs_objectifs([matb], [modularite], labels, dparamats=None)

In [ ]:
pddat

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(matb)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/emulee_obs1.pdf")

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(mat)
plt.tick_params(labelsize=1)
plt.tight_layout()
#plt.savefig("images/emulee_theo.pdf")

### Experience 2

Expérience dont les tests de présence de goulot ont été présentés dans l'article. Des files d'attente en amont rendent les goulots d'étranglements un peu plus difficiles à identifier que dans l'expérience 1.

In [ ]:
matb=np.load('donnees/experience2.npy')
blocliste=[0, 72, 24, 72]
mat=construit(blocliste)#matrice théorique
labels=labelblocs(blocliste)

In [ ]:
#résolution avec la méthode spectrale et la modularité. Le calcul de la vraisemblance n'est pas possible car les paramètres sont inconnus
pddat = teste_foncs_objectifs([matb], [modularite], labels, dparamats=None)

In [ ]:
pddat

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(matb)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/emulee_obs2.pdf")

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(mat)
plt.tick_params(labelsize=1)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("images/emulee_theo.pdf")

## Estimation des paramètres de la matrice

In [ ]:
# estimation des listes de paramètres p_n, q_n de la dernière matrice matb à partir de la somme des résultats
# Cette méthode peut être optimisée en estimant un maximum de vraisemblance. 
# Le coût sera probablement bien plus élevé que résoudre ce système de type Ax=b sous contrainte.
somlignp = np.sum(mat*matb, axis=1)
matlignp = mat/2
matlignp[np.diag_indices_from(matlignp)] = (np.sum(mat, axis=1)+1)/2
amat = (1-mat)
somlignq = np.sum(amat*matb, axis=1)
matlignq = amat/2+np.diag(np.sum(amat, axis=1))/2

eps=1e-10#pour gérer les erreurs de log
qns = cp.Variable(len(matlignq))
pns = cp.Variable(len(matlignp))

objective = cp.Minimize(cp.sum_squares(matlignp @ pns - somlignp)+cp.sum_squares(matlignq @ qns - somlignq))
constraints = [eps <= pns, pns <= 1-eps, eps <= qns, qns <= 1-eps, qns<=pns]
prob = cp.Problem(objective, constraints)
result = prob.solve()

In [ ]:
#calcul des matrices paramètres associées aux p_n et q_n trouvés
dparamat=pmats(sum(blocliste), p=np.clip(pns.value, eps, 1-eps), q=np.clip(qns.value, eps, 1-eps))
paramat=pmat_labels(labels, *dparamat)

#exemple présenté dans l'article.
matb2=brmat(paramat, graine=13)
pddat = complet([matb2], [mod_bloc, vraiss], labels, dparamats=[dparamat])

In [ ]:
#autres réalisations de matrices d'adjacences associées au modèle par nœuds estimé. Prend un peu de temps.
#pddat = complet([brmat(paramat, graine=i) for i in range(14)], [modularite, vraiss], labels, dparamats=[dparamat])

In [ ]:
plt.figure(figsize=(5,5.2))
plt.imshow(matb2, vmin=0, vmax=1)
plt.tick_params(labelsize=16)
plt.xlabel('indices des nœuds', fontsize=16)
plt.tight_layout()
#plt.savefig("/home/paul/Documents/paul-grislain/rédaction/gretsi/images/simule_expe2.pdf")